# [03] 모델 만들기1 - BaseConv

우리가 만들고자 하는 객체 검출기(Object Detector)는 3개의 부분으로 구성됩니다. 각 모듈들의 역할은 다음과 같습니다.

- BaseConv: 이미지 데이터의 특징(Feature)을 추출
- AuxConv: BaseConv로부터의 특징을 다양한 스케일로 학습
- PredConv: 특징들을 기반으로 Bounding Box와 ClassScore를 예측

먼저, 첫번째 모듈인 `BaseConv`를 만들어 보겠습니다. BaseConv에는 여러 CNN 아키텍쳐(VGG, ResNet, DenseNet, Wide-ResNet, MobileNet ... 등)를 사용할 수 있지만, 이번 실습에서는 모델의 크기와 연산량 면에서 효율적인 EfficientNet을 활용해 보도록 하겠습니다.

---------------
EfficientNet은 MobileNet구조를 기반으로 아키텍쳐를 탐색한 다음, Width, Depth, Resolution을 복합적으로 고려해 모델을 Scaling했을 때, 큰 성능 향상을 기대할 수 있다는 것을 제안하였습니다. 

- 아래는 EfficientNet의 Compund Scaling 기법과 성능에 대해 나타내고 있습니다. 더블클릭을 통해 확대해 살펴보세요.

In [1]:
from IPython.display import HTML, display

# Images from https://arxiv.org/abs/1905.11946
display(HTML("<table><tr><td><img src='img/[03]eff_arch.png'></td><td><img src='img/[03]eff_perf.png'></td></tr></table>"))

,


-------------------
## [Task 1] Pre-trained EfficientNet

`materials.models`에 EfficientNet의 코드가 있습니다. 이 코드를 활용하여, ImageNet에 대해 pre-trained된 EfficientNet-b0의 weight를 가져오도록 하겠습니다.

In [2]:
from materials.models.EfficientNet import EfficientNet
from materials.utils import *

model = EfficientNet.from_pretrained('efficientnet-b0')

# 아래 주석을 해제하여 efficientnet-bo의 구조를 살펴보세요!
# print(model)

Loaded pretrained weights for efficientnet-b0


원래의 모델은 imagenet 1000개의 class에 대한 예측치를 출력하도록 학습되었지만, Object Detection을 위해서는 예측치 대신 다양한 크기의 featuremap이 필요하기 때문에 16개의 block중에서 11번째, 16번째 block에서의 featuremap을 리턴하도록 바꾸어 놓은 상태입니다.

### ToDo: 모델의 출력값 확인하기

우리가 사용할 300x300의 이미지에 대하여 모델이 어떤 형태의 출력값을 가지는지 확인해 봅시다.

In [3]:
# [ToDo]: 300 * 300 의 random tensor를 모델에 입력해 결과를 출력합니다.
sample = ??????????????????
outputs = model(sample)

for i, out in enumerate(outputs):
    print('{}번째 리턴값의 Shape: {}'.format(i+1, out.shape))

1번째 리턴값의 Shape: torch.Size([4, 112, 18, 18])
2번째 리턴값의 Shape: torch.Size([4, 1280, 9, 9])


------------
## [Task 2] Selectively Freeze Layers

학습된 weights를 최대한 활용하기 위해서, pre-trained model의 weight를 학습이 되지 않는 상태로 만들고자 합니다. 

그러나 학습 도메인의 데이터 또한 모델이 충분히 학습할 수 있기를 기대하기 때문에, 모델의 일부는 학습이 가능한 형태로 남겨두고 싶습니다.

### ToDo: Freeze / Unfreeze Layers

- 모델 전체를 학습이 되지 않도록 설정합니다.
- 모델의 block15의 parameter를 학습가능하도록 설정합니다. (_block15)
- 모델의 convhead의 parameter를 학습가능하도록 설정합니다. (_convhead)
- 모델의 마지막 batchnorm 층의 parameter를 학습가능하도록 설정합니다. (_bn1)

In [4]:
model = EfficientNet.from_pretrained('efficientnet-b0')

# [ToDo]: 모델의 모든 Parameter를 학습되지 않도록 설정합니다.
for params in model.parameters():
    params.?????????? = ???????
    
# [ToDo]: block15의 parameter와 convhead의 parameter는 학습이 가능하도록 합니다.
for name, params in model.named_parameters():
    if str(15) in str(name):
        params.requires_grad = True
            
    if 'head' in str(name):
        params.requires_grad = True
        
    if '_bn1' in  str(name):
        params.requires_grad = True

Loaded pretrained weights for efficientnet-b0


In [5]:
# hint : 모델의 parameter들에는 이름이 있습니다!
#for name, params in model.named_parameters():
#    param_names.append(name)
#    print(name)

------------
## [Task 3] BaseConvluition Class

Task1, Task2를 통해 객체 검출(Object Detection)을 위한 특징 추출기(feature extractor)를 만들었습니다. 지금까지의 과정을 통합하여 pytorch의 `nn.module` Class를 상속하는 하나의 Neural Network 모델로 구성해 봅시다.

### ToDo: nn.Module Class로 통합하기

- `__init__()` 함수를 완성합니다.
- `forward` 함수를 완성합니다.

In [6]:
import torch
import torch.nn as nn
from materials.models.EfficientNet import EfficientNet

class BaseConvolution(nn.Module):
    def __init__(self, unfreeze_keys=['15', 'head', 'bn1']):
        """
        Pre-trained EfficientNet을 불러와 BaseConv를 구성합니다,
        """        
        super(BaseConvolution, self).__init__()
        self.base = EfficientNet.from_pretrained('efficientnet-b0')
        
        # 모델의 모든 parameter를 학습이 되지 않도록 합니다.
        for params in self.base.parameters():
            params.requires_grad = False
        
        # [ToDo]: `unfreeze_keys`에 해당하는 parameter는 학습이 가능하도록 합니다.
        for name, params in self.base.named_parameters():
            for key in ????????????:
                if key in ???????????:
                    params.??????????? = ?????????
        
        
    def forward(self, x):
        """
        BaseConvolution을 통과하여 나온 3개의 featuremap을 리턴합니다. 
        (tensor) x: (N, C, H, W)의 Image Data.
        """
        # base conv 연산을 수행합니다.
        mid, end = self.base(x)
        
        return mid, end
    

In [7]:
# BaseConvolution Class를 테스트합니다.
baseconv = BaseConvolution()

sample = torch.randn(4, 3, 300, 300)
mid, end = baseconv(sample)

print('\nmid shape {}'.format(mid.shape))
print('end shape {}\n'.format(end.shape))

Loaded pretrained weights for efficientnet-b0

mid shape torch.Size([4, 112, 18, 18])
end shape torch.Size([4, 1280, 9, 9])



---------
### <생각해 봅시다>

- Feature map을 몇 번째 block에서 뽑는지는 어떤 영향을 미칠까요?
- Pre-trained model의 freeze/unfreeze의 여부를 어떻게 정해야 할까요?

------------